In [ ]:
%pip install geopandas

Iz .gml datoteke koja sadrži podatke o koordinatama granica raznih geografskih jedinica u Hrvatskoj, izvući ćemo samo ono što nam je bitno(granice gradova i općina, te županija), i te podatke spremiti u odgovarajuće .geojson datoteke.

In [ ]:
import geopandas as gpd

# Input GML file
input_gml = "data/administrative_units/AdministrativeUnit.gml"

# Load the data into a GeoDataFrame
gdf = gpd.read_file(input_gml)

# Filter rows by attribute 
gdf_gradovi_opcine = gdf[gdf["LocalisedCharacterString"] == "Jedinica lokalne samouprave"]
gdf_zupanije = gdf[gdf["LocalisedCharacterString"] == "Županija"] 

# Imena županija ćemo pretvoriti u velika slova jer su ovdje mala, a u podatcima o izborima su velika
gdf_zupanije.loc[:, "text"] = gdf_zupanije["text"].str.upper()

# Promijeniti ćemo default projekciju na karti
gdf_gradovi_opcine = gdf_gradovi_opcine.to_crs(epsg=3765)
gdf_zupanije = gdf_zupanije.to_crs(epsg=3765)

# Output GeoJSON file
output_geojson_gradovi_opcine = "data/administrative_units/gradovi_opcine.geojson"
output_geojson_zupanije = "data/administrative_units/zupanije.geojson"

# Save the filtered data as GeoJSON
gdf_gradovi_opcine.to_file(output_geojson_gradovi_opcine, driver="GeoJSON")
gdf_zupanije.to_file(output_geojson_zupanije, driver="GeoJSON")


Trenutno, gradovi_opcine.geojson ne sadrži podatke o tome u kojoj se županiji nalazi grad/općina. To je problem jer ima nekoliko gradova/općina s istim imenom. Zato ćemo napraviti spatial join, koji će gradu/općini pridijeliti županiju na temelju podataka o koordinatama granica gradova/općina i županija.

In [ ]:
gdf = gpd.sjoin(gdf_gradovi_opcine, gdf_zupanije, how="left", predicate="within")

gdf.to_file("data/administrative_units/gradovi_opcine_zupanije.geojson", driver="GeoJSON")

Možemo prikazati gradove/općine na karti.

In [ ]:
%pip install matplotlib

In [ ]:
gdf.plot(figsize=(10,10))

Možemo prikazati i interaktivnu kartu.

In [ ]:
%pip install folium
%pip install mapclassify

In [ ]:
gdf.explore()